
### Exercício 1. 

Use a package Cryptography   e  o package ascon (instalar daqui) para  criar um comunicação privada assíncrona em modo  “Lightweight Cryptography” entre um agente Emitter e um agente Receiver que cubra os seguintes aspectos:
1.  Autenticação do criptograma e dos metadados (associated data) usando Ascon (ver implementação aqui) em modo de cifra.
2.  As chaves de cifra, autenticação  e  os “nounces” são gerados por um gerador pseudo aleatório (PRG)  usando o Ascon em modo XOF. As diferentes chaves para inicialização do PRG são inputs do emissor e do receptor.
3.  Para implementar a comunicação cliente-servidor use o package python `asyncio`. 

Usámos o pacote Ascon para autenticar o criptograma envolvido na comunicação bem como os metadados envolventes. O Ascon é uma família de algoritmos de encriptação autenticada e de hashing concebidos para serem leves e fáceis de implementar, mesmo com contramedidas adicionais contra ataques de canais laterais. Foi concebido por uma equipa de criptógrafos da Universidade de Tecnologia de Graz, da Infineon Technologies e da Universidade de Radboud: Christoph Dobraunig, Maria Eichlseder, Florian Mendel e Martin Schläffer.

O NIST anunciou a seleção da família Ascon como o padrão de criptografia leve em fevereiro de 2023, após receber feedback público num workshop. O NIST está a trabalhar com a equipa Ascon para elaborar os padrões de criptografia leve.

Asyncio é uma biblioteca para escrever código concorrente usando a sintaxe async/await, por isso tirámos proveito dessa funcionalidade para poder implementar a comunicação cliente-servidor.

In [9]:
import asyncio
import os
import ascon
from cryptography.hazmat.primitives.asymmetric import x25519
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from secrets import token_bytes

A classe AsconCipher é uma classe que encapsula operações de criptografia e descriptografia Ascon. Tem como métodos:
1. encrypt: Usa o algoritmo Ascon para criptografar o texto simples.
2. decrypt: Usa o algoritmo Ascon para descriptografar o texto cifrado.
   
Utilizam o Ascon-128.

In [10]:
class AsconCipher:
    def __init__(self, key):
        self.key = key

    def encrypt(self, nonce, plaintext, associated_data):
        return ascon.ascon_encrypt(self.key, nonce, associated_data, plaintext, variant='Ascon-128')

    def decrypt(self, nonce, ciphertext, associated_data):
        return ascon.ascon_decrypt(self.key, nonce, associated_data, ciphertext, variant='Ascon-128')

A função handle_client é uma função assíncrona que lida com a comunicação com um cliente. Usa X25519 (algoritmo de Diffie-Hellman) para troca de chaves, deriva chaves com HKDF (baseado em HMAC) e realiza criptografia/descriptografia com Ascon.
A comunicação envolve a troca de chaves públicas, o estabelecimento de um segredo partilhado e a criptografia/descriptografia de mensagens.

In [11]:
async def handle_client(reader, writer):
    private_key = x25519.X25519PrivateKey.generate()
    public_key = private_key.public_key().public_bytes(serialization.Encoding.Raw, serialization.PublicFormat.Raw)

    writer.write(public_key)
    await writer.drain()

    peer_public_key_bytes = await reader.read(32)
    peer_public_key = x25519.X25519PublicKey.from_public_bytes(peer_public_key_bytes)

    shared_key = private_key.exchange(peer_public_key)

    # Derive keys using HKDF
    derived_key_material = HKDF(
        algorithm=hashes.SHA256(),
        length=64,
        salt=None,
        info=b'handshake data',
        backend=default_backend()
    ).derive(shared_key)

    cipher_key = derived_key_material[:32]

    cipher = AsconCipher(cipher_key)

    while True:
        nonce = await reader.readexactly(16)
        ciphertext = await reader.readuntil(separator=b'|')
        tag = await reader.readexactly(16)
        associated_data = await reader.readuntil(separator=b'||')
        plaintext = cipher.decrypt(nonce, ciphertext, tag, associated_data)

        writer.write(plaintext)
        await writer.drain()

A função main é a função principal e configura e executa o servidor.

In [12]:
async def main():
    server = await asyncio.start_server(
        handle_client, '127.0.0.1', 8888)

    addr = server.sockets[0].getsockname()
    print(f'Serving on {addr}')

    async with server:
        await server.serve_forever()

A função send_public_key é a função responsável por enviar chaves públicas do cliente para o servidor.

É gerada uma chave privada X25519 para o cliente. É obtida a chave pública correspondente à chave privada gerada e converte-a para bytes usando o formato de codificação especificado. A chave pública é escrita no objeto de escrita associado ao servidor. Por fim, espera que os dados sejam realmente gravados no socket, garantindo que a escrita seja concluída antes de continuar.

In [13]:
async def send_public_key(writer):
    private_key = x25519.X25519PrivateKey.generate()
    public_key = private_key.public_key().public_bytes(serialization.Encoding.Raw, serialization.PublicFormat.Raw)
    writer.write(public_key)
    await writer.drain()

A função send_message é a função responsável por enviar mensagens do cliente para o servidor.

Cria um loop infinito para que o cliente possa enviar várias mensagens consecutivas. Solicita ao utilizador que insira uma mensagem a ser enviada ao servidor. Verifica se o utilizador digitou 'quit' para encerrar o loop.
É então aberta uma conexão com o servidor, a função send_public_key é chamada para enviar a chave pública antes da mensagem, que depois é codificada para bytes e escrita no objeto de escrita associado ao servidor.
A função writer.drain() espera que os dados sejam realmente gravados no socket antes de continuar.
Quando gravados, são lidos até 1024 bytes de dados do servidor que exibe a mensagem recebida decodificando os bytes usando UTF-8.
A conexão com o servidor é fechada após o envio e receção da mensagem.

A função except KeyboardInterrupt captura a exceção de interrupção do teclado (Ctrl+C) para permitir a saída controlada do cliente.

In [14]:
async def send_message():
    while True:
        try:
            message = input("Enter message to send (or 'quit' to exit): ")
            if message.lower() == 'quit':
                break
            
            reader, writer = await asyncio.open_connection('127.0.0.1', 8888)

            # Send public key
            await send_public_key(writer)

            # Encrypt and send message
            writer.write(message.encode())
            await writer.drain()

            # Receive and decrypt response
            data = await reader.read(1024)  # Increased buffer size for receiving data
            print(f"Received message from server: {data.decode('utf-8', 'ignore')}")  # Decode using UTF-8

            writer.close()
            await writer.wait_closed()
        except KeyboardInterrupt:
            print("Exiting client...")
            break

A task cria e executa uma tarefa assíncrona para o servidor (main).

In [15]:
task = asyncio.create_task(main())

Task exception was never retrieved
future: <Task finished name='Task-5' coro=<main() done, defined at C:\Users\bruna\AppData\Local\Temp\ipykernel_22060\3117860583.py:1> exception=OSError(10048, "error while attempting to bind on address ('127.0.0.1', 8888): normalmente só é permitido uma utilização de cada endereço de socket (protocolo/endereço de rede/porta)")>
Traceback (most recent call last):
  File "C:\Users\bruna\AppData\Local\Temp\ipykernel_22060\3117860583.py", line 2, in main
    server = await asyncio.start_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2288.0_x64__qbz5n2kfra8p0\Lib\asyncio\streams.py", line 84, in start_server
    return await loop.create_server(factory, host, port, **kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2288.0_x64__qbz5n2kfra8p0\Lib\asyncio\base_events.py", line 1536

Por fim, é criada e executada uma tarefa assíncrona para enviar mensagens ao servidor (send_message).

In [16]:
await asyncio.create_task(send_message())

Received message from server: -%)2g%w#y9w+#
Received message from server: laU^h,>^	Y ᕩx"
Received message from server: N8ZqMKҦ6/w62]
ЀZ,ؗ5
Received message from server: oޑg鴸"zpش'1YrE
Received message from server: @;xU,`==X֜JM":G
Received message from server: hnKꛩ-۪ ^[
Received message from server: K?o6A%rѹ"y#;Š]nn
Received message from server: {E,zt9/>wEʩYEb
